# Neural Networks - MNIST Dataset

## Table of contents
2. [Introduction](#Introduction)
3. [Import the MNIST dataset](#Import-the-MNIST-dataset)
    1. [Download the dataset locally](#Download-the-dataset-locally)
    2. [Load the MNIST dataset](#Load-the-MNIST-dataset)
4. [Define the Neural Network Model](#Define-the-Neural-Network-Model)
5. [Train the Neural Network Model](#Train-the-Neural-Network-Model)
6. [Conclusion](#Conclusion)

## Introduction
This notebook examines how the neural network algorithm operates in more detail. The neural network algorithm is applied to the MNIST handwriting dataset with the internal operations defined fully below. The defined algorithm is not the most optimal algorithm in terms of execution time, however, it is very useful in terms of helping to understand the inner workings of the algorithm. 

This notebook primarily follows the basic neural network code as defined by Micheal Nelson in his online pdf `http://neuralnetworksanddeeplearning.com/`. This book was very useful in helping me to initially understand the internal operations of a neural network model.

## Import the MNIST dataset

In [3]:
# load the necessary libraries
import os
import random  
import gzip
import pickle
import urllib.request
import shutil

In [4]:
# define the absolute root path
ROOT_PATH = os.getcwd()

### Download the dataset locally

In [5]:
# create a location to store the dataset
try:
    os.stat(ROOT_PATH + '/data')
except:
    os.mkdir(ROOT_PATH + '/data') 

In [6]:
# define the file location and path to store the MNIST dataset
url = 'http://deeplearning.net/data/mnist/mnist.pkl.gz'
file_name = ROOT_PATH + '/data/mnist.pkl.gz'

In [7]:
# download the file from url and save it locally under file_name:
# (this may take a few minutes)
with urllib.request.urlopen(url) as response, open(file_name, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

### Load the MNIST dataset

In [8]:
# define the library which loads the MNIST image dataset
def load_data():
    """
    The load_data function returns the MNIST data as a tuple 
    containing the training, validation and test datasets.

    The training_data is returned as a tuple with two entries.
    The first entry contains the actual training images. This is 
    a numpy.ndarray with 50,000 entries. Each entry is, in turn, 
    a numpy.ndarray with 784 values, representing the 28*28 = 784
    pixels in a single MNIST image.

    The second entry in the training_data tuple is a numpy.ndarray
    containing 50,000 entries. Those entries are just the digit
    values (0,...,9) for the corresponding images contained in the 
    first entry of the tuple.

    The validation_data and test_data are similar, except each 
    contains only 10,000 images, respectively.
    """
    
    # open the tar file object
    f = gzip.open(ROOT_PATH + '/data/mnist.pkl.gz', 'rb')
    
    # extract the training, validation and test datasets
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
    
    # close the tar file object
    f.close()
    
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """
    The load_data_wrapper function returns a tuple containing 
    (training_data, validation_data, test_data). The function 
    first loads the training, validation and test datasets 
    using the load_data function.

    The training_data is transformed into a list containing 
    50,000 2-tuples (x, y). x is a 784-dimensional 
    numpy.ndarray containing the input image. y is a 
    10-dimensional numpy.ndarray representing the unit vector 
    corresponding to the correct digit for x.

    The validation_data and test_data datasets are transformed 
    into lists containing 10,000 2-tuples (x, y). In each case, 
    x is a 784-dimensional numpy.ndarray containing the input 
    image, and y is the corresponding classification, i.e., the 
    digit values (integers) corresponding to x.

    Therefore, the formats of the training data and the 
    validation/test data are different, however these formats 
    turn out to be the most convenient for use in our neural 
    network code.
    """
    
    # import the training, validation and test datasets
    tr_d, va_d, te_d = load_data()
    
    # reshape the training input images into 784 dimensional numpy arrays
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    
    # create a classification vector where each classification value is assigned a 1, other values 0
    training_results = [vectorized_result(y) for y in tr_d[1]]
    
    # combine the inputs and classification results into a list of tuples for each pair of x,y values 
    training_data = list(zip(training_inputs, training_results))
    
    # reshape the input images into 784 dimensional numpy arrays
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    
    # combine the input images and classification results into a list of tuples for each pair of x,y values
    # this time keep the original classification values
    validation_data = list(zip(validation_inputs, va_d[1]))
    
    # reshape the input images into 784 dimensional numpy arrays
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    
    # combine the input images and classification results into a list of tuples for each pair of x,y values
    # this time keep the original classification values
    test_data = list(zip(test_inputs, te_d[1]))
    
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """
    Return a 10-dimensional unit vector with a 1 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0,...,9) into a corresponding desired output from the neural
    network.
    """
    
    # create a vector of 0s
    e = np.zeros((10, 1))
    
    # replace the jth value with a 1
    e[j] = 1.0
    
    return e

In [9]:
# generate the training, validation and test datasets
training_data, validation_data, test_data = load_data_wrapper()

## Define the Neural Network Model

In [10]:
# Define the network class which implements the stochastic gradient descent learning algorithm 
# for a feedforward neural network. Gradients are calculated using backpropagation. 

class Network(object):
    
    # initalise the neural network    
    def __init__(self, sizes):
        """
        The sizes list contains the number of neurons in the
        respective layers of the network. For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian distribution 
        with mean 0, and variance 1. Note that the first layer is 
        assumed to be an input layer, and by convention we won't set any 
        biases for those neurons, since biases are only ever used in 
        computing the outputs from later layers.
        """
        
        # define the number of layers
        self.num_layers = len(sizes)
        
        # define the number of neurons in each layer        
        self.sizes = sizes

        # randomly choose the initial weights and biases
        # define the biases associated to each neuron in all layers excluding the input layer
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        
        # define the weights, where the weights connect the neurons from two seperate layers
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        
    
    # define the feedforward function which takes inputs and multiplies them by the weights adds the bias before 
    # finally computing the the sigmoid of these values
    def feedforward(self, a):
        """
        Return the output of the network if a is input.
        """
        
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    # define the stochastic gradient descent function
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """
        Train the neural network using mini-batch stochastic
        gradient descent. The training_data is a list of tuples
        (x, y) representing the training inputs and the desired
        outputs. The other non-optional parameters are self 
        explanatory. If test_data is provided then the network 
        will be evaluated against the test data after each epoch, 
        and partial progress printed out. This is useful for 
        tracking progress, but slows things down substantially.
        """
        
        # if given a test dataset calculate the number of observations in the test dataset
        if test_data: 
            n_test = len(test_data)
            
        # define the number of training observations
        n = len(training_data)
        
        # for each epoch
        for j in range(epochs):
            
            # randomly shuffle the training dataset
            random.shuffle(training_data)
            
            # split the training dataset up into equal mini_batch_size sized mini-batches  
            # we have a list of mini-batches of len mini_batch_size each containing mini_batch_size
            # tuples of x and y values
            mini_batches = [training_data[k:k+mini_batch_size]
                            for k in range(0, n, mini_batch_size)]
            
            # for each mini-batch apply a single step of gradient descent
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            
            # evaluate the test data results at the end of each epoch
            if test_data:
                print("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))
            
            else:
                print("Epoch {0} complete".format(j))
    
    # update the weights and biases of the network (after single iteration)
    def update_mini_batch(self, mini_batch, eta):
        """
        Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini-batch.
        The mini_batch is a list of tuples (x, y), and eta is the 
        learning rate.
        """
        
        # define empty vectors to store the changes in the weights and biases
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # for each tuple of x and y in the mini-batch
        for x, y in mini_batch:
            
            # call the backpropagation algorithm and calculate the gradient of the cost function at x (i.e. C_x)
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            
            # update the weights and the biases (add the changes across all tuples in the mini-batch)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            
        # update the weights of each layer
        self.weights = [w - (eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        
        # update the biases of each layer
        self.biases = [b - (eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """
        Return a tuple (nabla_b, nabla_w) representing the
        gradient for the cost function C_x. nabla_b and
        nabla_w are layer-by-layer lists of numpy arrays, similar
        to self.biases and self.weights.
        """
        
        # define empty vectors to store the changes in the weights and biases
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # feedforward
        
        # define the x the activation input, the 784 dimensional vector that is inputted into the network
        activation = x
        
        # list to store all the activations, layer by layer
        # add the input layer to the list
        activations = [x] 
        
        # list to store all the activation function input z vectors, layer by layer
        zs = []
        
        # for the biases and weights of each layer
        for b, w in zip(self.biases, self.weights):
            
            # calculate the activation input z for each neuron in the layer
            z = np.dot(w, activation) + b
            
            # add the activation input z to the zs list
            zs.append(z)
            
            # tranform the activation inputs for the layer through the sigmoid function
            # each neurons z value is transformed individually
            activation = sigmoid(z)
            
            # add the activation output to the activations list
            activations.append(activation)
        
        
        # backward pass
        
        # calculate delta^L = nabla_a_C * sigma_prime(z^L)
        # find the output error for the final layer
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        
        # remember that the change in the cost relative to the change in the biases is equal to delta
        nabla_b[-1] = delta
        
        # remember that the change in the cost relative to the change in the weights is equal to delta * activations 
        # of the previous layer
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book. Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on. It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        
        # for the previous layers (the second layer)
        for layer in range(2, self.num_layers):
            
            # take the activation input for the second last layer 
            z = zs[-layer]
            
            # pass the activation input through the derivative of the activation function
            sp = sigmoid_prime(z)
            
            # pass the error backwards (from the last layer to the second last layer)
            # delta^L = (w^(L+1))^T * delta^(L+1) * sigma_prime(z^L)
            delta = np.dot(self.weights[-layer+1].transpose(), delta) * sp
            
            # again set the change in the cost relative to the biases to be the error delta
            nabla_b[-layer] = delta
            
            # set the change in the cost relative to the weights to be the dot product of the previous error 
            # and activations from the current layer
            nabla_w[-layer] = np.dot(delta, activations[-layer-1].transpose())
            
        return (nabla_b, nabla_w)
    
    # evaluate the currrent model on the test dataset
    def evaluate(self, test_data):
        """
        Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation.
        """
        
        # pass the unseen x values through the feedforward algorithm
        # argmax finds the index of the max value of the output of 
        # the final layer
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        
        # return the total number of matchs between y and those predicted by the NN
        return sum(int(x == y) for (x, y) in test_results)
    
    # define the derivative of the cost function
    def cost_derivative(self, output_activations, y):
        """
        Return the vector of partial derivatives \partial C_x / \partial a 
        for the output activations. This is the derivative of the quadratic 
        cost function.
        """
        return (output_activations-y)

# activation function
# define the sigmoid activation function
def sigmoid(z):
    """The sigmoid function."""
    
    return 1.0/(1.0+np.exp(-z))

# define the derivative of the sigmoid activation function
def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    
    return sigmoid(z)*(1-sigmoid(z))

## Train the Neural Network Model

In [11]:
# define a 3 layered network, the initial layer containing 784 (28*28) neurons, the middle hidden layer contains 30 
# neurons, while the output layer contains 10 neurons
net = Network([784, 30, 10])

In [12]:
# train the network
# (this might take some time)
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 8271 / 10000
Epoch 1: 8360 / 10000
Epoch 2: 8449 / 10000
Epoch 3: 8499 / 10000
Epoch 4: 8493 / 10000
Epoch 5: 9343 / 10000
Epoch 6: 9379 / 10000
Epoch 7: 9426 / 10000
Epoch 8: 9408 / 10000
Epoch 9: 9441 / 10000
Epoch 10: 9442 / 10000
Epoch 11: 9476 / 10000
Epoch 12: 9476 / 10000
Epoch 13: 9471 / 10000
Epoch 14: 9489 / 10000
Epoch 15: 9521 / 10000
Epoch 16: 9498 / 10000
Epoch 17: 9504 / 10000
Epoch 18: 9509 / 10000
Epoch 19: 9521 / 10000
Epoch 20: 9516 / 10000
Epoch 21: 9512 / 10000
Epoch 22: 9451 / 10000
Epoch 23: 9498 / 10000
Epoch 24: 9475 / 10000
Epoch 25: 9508 / 10000
Epoch 26: 9488 / 10000
Epoch 27: 9481 / 10000
Epoch 28: 9492 / 10000
Epoch 29: 9511 / 10000


## Conclusion

The main aim of this notebook was to delve deeper into the Neural Network algorithm. TensorFlow and in particular Keras both provide a very straightforward entry point when applying Neural Network models to data. However, oftentimes the user may not actually understand the algorithm itself and how it operates. This is where this notebook is very important as I believe it helps a user fully understand how the theory behind a neural network which in my opinion is very important.

Examining the results of the algorithm one can see that after 30 epochs the neural network algorithm obtained an overall test classification accuracy of 95%.